In [19]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import time
from collections import defaultdict

In [46]:
def get_cleaning_fee(price_item_list):
    """Applies a filter to get the pricing item that is the cleaning."""
    def cleaning_fee_filter(item):
        return item['localizedTitle'] == 'Cleaning fee'
    cleaning_fee_item = list(filter(cleaning_fee_filter, price_item_list))
    if cleaning_fee_item:
        return cleaning_fee_item[0]['total']['amount']
    else:
        return 0
                             
# stays[0]['pricingQuote']['price']['priceItems']))[0]['total']['amount']

In [2]:
def setup_webdriver(width=800, height=600):
    """Opens a chrome window and sets the size of the window. The webdriver object is returned."""
    driver = webdriver.Chrome()
    driver.set_window_size(width, height, windowHandle='current')
    time.sleep(1)
    return driver

In [3]:
def get_stays_on_page(page_source):
    """Use BeautifulSoup to extract the stays on a page. Airbnb uses a json object
    that contains much of the desired information."""
    soup = BeautifulSoup(PageSourceURL)
    js = soup.select('script#data-state')
    listings = json.loads(js[0].string)

In [4]:
driver = setup_webdriver(1730, 1020)

In [5]:
driver.switch_to.window(driver.window_handles[0])
link = 'https://www.airbnb.com/s/Bogota--Colombia/homes?tab_id=all_tab&refinement_paths%5B%5D=%2Fhomes&query=Bogota%2C%20Colombia&place_id=ChIJKcumLf2bP44RFDmjIFVjnSM&checkin=2020-06-14&checkout=2020-06-28&source=structured_search_input_header&search_type=search_query'
# #link = 'https://www.airbnb.com/rooms/13516185?location=Bogota%2C%20Colombia&check_in=2020-06-14&check_out=2020-06-28&source_impression_id=p3_1590372927_i1WyHyNErr07fsxz&guests=1&adults=1'
driver.get(link)
time.sleep(5)
# WebDriverWait(driver,20).until(EC.presence_of_element_located((By.CSS_SELECTOR,'_fhph4u')))
# WebDriverWait(driver,20).until(EC.presence_of_element_located((By.CSS_SELECTOR,'data-state')))
PageSourceURL = driver.page_source

In [6]:
# <script data-state="true" id="data-state" type="application/json">

In [7]:
soup = BeautifulSoup(PageSourceURL)
js = soup.select('script#data-state')
listings = json.loads(js[0].string)

In [8]:
# JSON {}
# niobeClinetData
# __niobe_denormalized {}
# 0 []
# 1 {}
# dora {}
# ExplorerV3
# sections []
# 1 {}
# Items

In [9]:
stays = listings['niobeClientData']['__niobe_denormalized']['queries'][0][1]['dora']['exploreV3']['sections'][1]['items']
len(stays)

20

In [20]:
# stays[0]

In [47]:
list(filter(filter_cleaning_fee, stays[0]['pricingQuote']['price']['priceItems']))[0]['total']['amount']

15

In [48]:
stay_dict = defaultdict(list)
for stay in stays:
    stay_dict['guests'].append(stay['listing']['personCapacity'])
    stay_dict['bedrooms'].append(stay['listing']['bedrooms'])
    stay_dict['beds'].append(stay['listing']['beds'])
    stay_dict['bathrooms'].append(stay['listing']['bathrooms'])
    stay_dict['sid'].append(stay['listing']['id'])
    stay_dict['amenities'].append(stay['listing']['amenityIds'])
    stay_dict['superhost'].append(stay['listing']['isSuperhost'])
    stay_dict['starRating'].append(stay['listing']['avgRating'])
    stay_dict['amount'].append(stay['pricingQuote']['price']['total']['amount'])
    stay_dict['cleaning_fee'].append(get_cleaning_fee(stay['pricingQuote']['price']['priceItems']))

In [50]:
# stay_dict